In [44]:
import argparse
import os.path
import time
import pandas as pd
import datetime
import configparser 
now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print (now)

2018-12-30 19:58


# Load Input CSV file

In [45]:
config = configparser.ConfigParser()
config.sections()
config.read('config.ini')
input_file=config['DEFAULT']['input_file']
print ('Input file:', input_file)

Input file: sample-imput/activities.csv


In [71]:
df_input_file=pd.read_csv(input_file, header=None, delimiter='|', names = ["orcid", "relation_type", "name", "source", "identifier", "file_name"])

In [72]:
df_input_file.describe()

,orcid,relation_type,name,source,identifier,file_name
count,1000,1000,1000,1000,1000,1000
unique,407,2,703,3,691,1000
top,0000-0001-5738-1893,educations,Universidad Central,RINGGOLD,27979,0000-0001-5123-8893_employments_4445793
freq,9,608,7,907,7,1


In [73]:
df_input_file.head()

,orcid,relation_type,name,source,identifier,file_name
0,0000-0002-1484-1893,employments,Instituto Politécnico do Porto Escola Superior...,RINGGOLD,203255,0000-0002-1484-1893_employments_5636083
1,0000-0002-1484-1893,educations,Universidade do Porto Faculdade de Desporto,RINGGOLD,224740,0000-0002-1484-1893_educations_5636056
2,0000-0002-1484-1893,educations,Universidade do Porto Faculdade de Desporto,RINGGOLD,224740,0000-0002-1484-1893_educations_5636066
3,0000-0002-1484-1893,educations,Instituto Politécnico do Porto Escola Superior...,RINGGOLD,203255,0000-0002-1484-1893_educations_5636079
4,0000-0001-5057-7893,employments,University of Oregon College of Education,RINGGOLD,143853,0000-0001-5057-7893_employments_64516


In [74]:
# Number of org IDs per source
df=df_input_file.groupby('source')['orcid'].count()
df

source
FUNDREF      32
GRID         61
RINGGOLD    907
Name: orcid, dtype: int64

In [75]:
# Number of unique org IDs per source
df=df_input_file.groupby('source')['identifier'].nunique()
df

source
FUNDREF      23
GRID         45
RINGGOLD    623
Name: identifier, dtype: int64

In [76]:
# Number of unique org name per source
df=df_input_file.groupby('source')['name'].nunique()
df

source
FUNDREF      23
GRID         45
RINGGOLD    636
Name: name, dtype: int64

# Create nodes_orcid_ringgold.csv

In [79]:
df_ringgold = df_input_file[df_input_file['source']=='RINGGOLD']
df_ringgold.head()

,orcid,relation_type,name,source,identifier,file_name
0,0000-0002-1484-1893,employments,Instituto Politécnico do Porto Escola Superior...,RINGGOLD,203255,0000-0002-1484-1893_employments_5636083
1,0000-0002-1484-1893,educations,Universidade do Porto Faculdade de Desporto,RINGGOLD,224740,0000-0002-1484-1893_educations_5636056
2,0000-0002-1484-1893,educations,Universidade do Porto Faculdade de Desporto,RINGGOLD,224740,0000-0002-1484-1893_educations_5636066
3,0000-0002-1484-1893,educations,Instituto Politécnico do Porto Escola Superior...,RINGGOLD,203255,0000-0002-1484-1893_educations_5636079
4,0000-0001-5057-7893,employments,University of Oregon College of Education,RINGGOLD,143853,0000-0001-5057-7893_employments_64516


In [82]:
#Select only three columns that we need.
df_ringgold = df_ringgold[['name','source','identifier']]
df_ringgold.head()

,name,source,identifier
0,Instituto Politécnico do Porto Escola Superior...,RINGGOLD,203255
1,Universidade do Porto Faculdade de Desporto,RINGGOLD,224740
3,Instituto Politécnico do Porto Escola Superior...,RINGGOLD,203255
4,University of Oregon College of Education,RINGGOLD,143853
8,Nanjing University,RINGGOLD,12581


In [83]:
# Remove any duplicate rows except the first one.
df_ringgold=df_ringgold.drop_duplicates(keep='first')
df_ringgold.head()

,name,source,identifier
0,Instituto Politécnico do Porto Escola Superior...,RINGGOLD,203255
1,Universidade do Porto Faculdade de Desporto,RINGGOLD,224740
3,Instituto Politécnico do Porto Escola Superior...,RINGGOLD,203255
4,University of Oregon College of Education,RINGGOLD,143853
8,Nanjing University,RINGGOLD,12581


In [96]:
df_ringgold['key']='researchgraph.org/ringgold/'+df_ringgold['identifier']
df_ringgold['source']='orcid.org'
df_ringgold['local_id']=df_ringgold['identifier']
df_ringgold['ringgold']=df_ringgold['identifier']
df_ringgold['last_updated']= now
df_ringgold.head()

,name,source,identifier,key,last_updated,local_id,ringgold
0,Instituto Politécnico do Porto Escola Superior...,orcid.org,203255,researchgraph.org/ringgold/203255,2018-12-30 19:58:56.945229,203255,203255
1,Universidade do Porto Faculdade de Desporto,orcid.org,224740,researchgraph.org/ringgold/224740,2018-12-30 19:58:56.945229,224740,224740
3,Instituto Politécnico do Porto Escola Superior...,orcid.org,203255,researchgraph.org/ringgold/203255,2018-12-30 19:58:56.945229,203255,203255
4,University of Oregon College of Education,orcid.org,143853,researchgraph.org/ringgold/143853,2018-12-30 19:58:56.945229,143853,143853
8,Nanjing University,orcid.org,12581,researchgraph.org/ringgold/12581,2018-12-30 19:58:56.945229,12581,12581


In [98]:
df_ringgold.to_csv('nodes_orcid_ringgold.csv', index=False, columns=['key','source','local_id','name', 'ringgold'], sep="|")